In [2]:
import tensorflow as tf
import facenet.src.facenet as facenet
from scipy import misc
from skimage import io
from skimage import transform
import numpy as np
import tqdm
import os

/home/vch/py3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
def get_filename(idx, folder='img_align_celeba/'):
    idstr = str(idx)
    fid = ''.join(['0'] * (6 - len(idstr)) + [idstr])
    return ''.join([folder, fid, '.jpg'])

def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0/np.sqrt(x.size))
    y = np.multiply(np.subtract(x, mean), 1/std_adj)
    return y  

def load_batch(ids):
    ims = []
    for idx in ids:
        im = io.imread(get_filename(idx))
        im = transform.resize(im, (160, 160), preserve_range=True)
        im = prewhiten(im)
        ims.append(im)
    return np.stack(ims)

In [4]:
sess = tf.Session()

In [29]:
saver = tf.train.import_meta_graph('models/model-20170512-110547.meta')

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


In [30]:
saver.restore(sess, 'models/model-20170512-110547.ckpt-250000')

INFO:tensorflow:Restoring parameters from models/model-20170512-110547.ckpt-250000


In [31]:
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")

In [32]:
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")

In [33]:
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

In [41]:
len(os.listdir('img_align_celeba/'))

202599

In [45]:
nums = np.arange(1, len(os.listdir('img_align_celeba')) + 1)
batch_size = 100

In [51]:
batch_embs = []
for left in tqdm.tqdm(range(0, len(nums), 100)):
    batch = load_batch(nums[left: left + 100])
    feed_dict = { images_placeholder: batch, phase_train_placeholder:False }
    embed = sess.run(embeddings, feed_dict=feed_dict)
    batch_embs.append(embed)
    
    

  0%|          | 0/2026 [00:00<?, ?it/s]/home/vch/py3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
100%|██████████| 2026/2026 [26:03<00:00,  1.30it/s]


In [52]:
embs_all = np.vstack(batch_embs)

In [53]:
batch_embs[0].shape

(100, 128)

In [54]:
embs_all.shape

(202599, 128)

In [1]:
import pickle

In [56]:
pickle.dump(embs_all, open('embeddings', 'wb'))

In [3]:
embs_all = pickle.load(open('embeddings', 'rb'))

In [6]:
with open('embeddings.txt', 'w') as f:
    f.write(str(embs_all.shape[0]) + '\n')
    f.write(str(embs_all.shape[1]) + '\n')
    for emb in embs_all:
        f.write(' '.join(map(str, emb)) + '\n')